<a href="https://colab.research.google.com/github/hugooleal/wanderai-imersao-alura/blob/main/Imers%C3%A3o_IA_Alura_WanderAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instala a biblioteca necessária para utilizar o Gemini
pip install -U -q google-generativeai

In [24]:
#Faz todas as importações necessárias para o código rodar todas as atividades
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get("API_KEY")
genai.configure(api_key=api_key)
import pandas as pd
import numpy as np
import pathlib
import textwrap
from IPython.display import display
from IPython.display import Markdown

#Cria uma função apenas para interpretar os caracteres de markdown nas respostas
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
#Disponibiliza a lista de modelos generativos
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
#Define as configurações do modelo generativa a serem usadas no projeto
configsList = [
    {
    "temperature": 0.5,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 8192,
},
{
    "temperature": 0.75,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 8192,
},
 {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 8192,
}
]


safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE",
    },
]

In [17]:
#Recolhe do usuário os inputs necessários para elaboração do roteiro de viagem
destino = input("Insira seu destino: ")
estiloViagem = input("Qual é o seu estilo de viagem? ")
duracaoViagem = input("Em qual período pretende viajar? Insira data de início e fim. ")
tiposAtividades = input("Quais os tipois de atividade que você mais gosta de fazer em uma viagem? ")
budget = input("Qual é o orçamento para sua viagem? Baixo, Médio ou Alto? ")

Insira seu destino: Amsterdam
Qual é o seu estilo de viagem? Agitada
Em qual período pretende viajar? Insira data de início e fim. 01/06/2024 a 07/06/2024
Quais os tipois de atividade que você mais gosta de fazer em uma viagem? Atividades ao ar livre, atividades noturnas, museus, arte e cultura
Qual é o orçamento para sua viagem? Baixo, Médio ou Alto? Médio


In [25]:
#Monta o prompt com base nas respostas do usuário e solicita 3 respostas diferentes, com as respectivas temperaturas: 0.5, 0.75 e 1
prompt = f"Você agora é um agente de viagens e possui vasta experiência sobre todos os países do mundo. Crie um roteiro de viagens para mim considerando as seguintes informações:\nDestino: {destino}\nEstilo de viagem: {estiloViagem}\nDuração da viagem: {duracaoViagem}\nTipos de atividade que mais gosto: {tiposAtividades}\nOrçamento disponível: {budget}\nOrientações para montar o roteiro:\n1. Monte o roteiro detalhado para cada dia da viagem. Se for preciso, pode consolidar até no máximo 3 dias.\n2. Leve em consideração a proximidade de cada atividade para montar os dias.\n3. Seja específico e detalhado sobre cada dia e atividade sugerida.\n4. Não repita atividades entre os dias.\n5. Dê dicas importantes, como por exemplo, se há necessidade de reserva antecipada para a atividade sugerida e inclua os links de reserva quando aplicável.\n6. Sugira opões de restaurantes em cada dia.\n7. Dê dicas sobre o melhor meio de se locomover na cidade.\n8. Informe a média do custo total da viagem com base no roteiro sugerido, em reais. Faça isso apenas ao final."

responses = []

for config in configsList:
  model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-latest",
    generation_config=config,
    safety_settings=safety_settings,
)

  response = model.generate_content(prompt)
  responses.append({'temperatura': str(config['temperature']), 'resposta':response.text})

print(responses)

[{'temperatura': '0.5', 'resposta': '## Roteiro de Viagem Agitada por Amsterdã: 01/06/2024 a 07/06/2024\n\n**Dia 1: Chegada e Imersão Cultural**\n\n* Chegada no Aeroporto Schiphol (AMS) e traslado para o hotel. Recomendo o transporte público, eficiente e econômico. Utilize o trem até a Estação Central de Amsterdã e, de lá, siga para o seu hotel.\n* **Tarde:** Explore o centro histórico a pé, admire os canais e as casas flutuantes. Visite a Casa de Anne Frank (reserve com antecedência: [https://www.annefrank.org/](https://www.annefrank.org/)), um local emocionante e importante para a história.\n* **Noite:** Jante no De Foodhallen, um mercado gastronômico com diversas opções deliciosas. Experimente as famosas batatas fritas holandesas no Manneken Pis, próximo ao mercado.\n* **Dica:** Adquira o "I amsterdam City Card" para transporte público ilimitado e entrada gratuita em diversos museus.\n\n**Dia 2: Arte, Bicicletas e Noite Vibrante**\n\n* **Manhã:** Alugue uma bicicleta (fácil de encon

In [26]:
#Cria um DataFrame com as repostas de cada temperatura
df = pd.DataFrame(responses)
df

,temperatura,resposta
0,0.5,## Roteiro de Viagem Agitada por Amsterdã: 01/...
1,0.75,## Roteiro Amsterdam Agitado - 01/06/2024 a 07...
2,1,## Roteiro de Viagem Agitada em Amsterdam (01/...


In [27]:
#Cria uma função para embedar as respostas de cada temperatura
def embedFunction(temperatura, resposta):
  return genai.embed_content(model="models/embedding-001",
                             content=resposta,
                             title=temperatura,
                             task_type="RETRIEVAL_DOCUMENT")['embedding']

In [28]:
#Aplica a funçã embedFunction no DataFrame
df['Embeddings'] = df.apply(lambda row: embedFunction(row['temperatura'], row['resposta']), axis=1)
df

,temperatura,resposta,Embeddings
0,0.5,## Roteiro de Viagem Agitada por Amsterdã: 01/...,"[0.032766566, -0.0568213, -0.007071082, 0.0263..."
1,0.75,## Roteiro Amsterdam Agitado - 01/06/2024 a 07...,"[0.039626263, -0.05295285, -0.014904089, 0.030..."
2,1,## Roteiro de Viagem Agitada em Amsterdam (01/...,"[0.051672146, -0.051463388, -0.0036041925, 0.0..."


In [29]:
#Cria uma função para conferir qual é a melhor resposta dentre as 3 com base no prompt inicial
def consultarMelhorResposta(consulta, base):
  embeddingConsulta = genai.embed_content(model="models/embedding-001",
                             content=consulta,
                             task_type="RETRIEVAL_QUERY")['embedding']

  produtosEscalares = np.dot(np.stack(base), embeddingConsulta)

  indice = np.argmax(produtosEscalares)

  return df.iloc[indice]['resposta']


In [30]:
#Retorna a melhor reposta com base na função consultarMelhorReposta
melhorResposta = consultarMelhorResposta(prompt, df['Embeddings'])
to_markdown(melhorResposta)

> ## Roteiro de Viagem Agitada em Amsterdam (01/06/2024 - 07/06/2024)
> 
> **Dia 1: Chegada & Cultura no Jordaan**
> 
> * Chegada no Aeroporto Schiphol (AMS) e deslocamento para o hotel. A forma mais eficiente é o trem (€4.30) até a Estação Central, e de lá, táxi ou transporte público até o seu hotel.
> * **Tarde:** Exploração do charmoso bairro Jordaan. Caminhe pelas ruelas, admire as casas típicas e explore as galerias de arte independentes, como a Galerie Petit.
> * **Jantar:** Moeders (tradicional holandês) ou Winkel 43 (famoso pela torta de maçã).
> * **Noite:**  Experimente a agitada vida noturna de Leidseplein. Paradiso, Melkweg e De School são ótimas opções com música ao vivo e DJs.
> 
> **Dia 2: História & Arte Intensa**
> 
> * **Manhã:** Anne Frank Huis (reserve com antecedência: [https://www.annefrank.org/](https://www.annefrank.org/)). Uma experiência emocionante e reflexiva.
> * **Almoço:**  Foodhallen, mercado gastronômico com diversas opções.
> * **Tarde:** Mergulho na arte de Van Gogh no Museu Van Gogh (reserve online: [https://www.vangoghmuseum.nl/](https://www.vangoghmuseum.nl/)).
> * **Jantar:** Proeflokaal 't Blauwe Theehuis (cozinha holandesa moderna) no Vondelpark.
> * **Noite:**  Curta um concerto no Concertgebouw, renomado por sua acústica impecável.
> 
> **Dia 3: Pedalando & Noites Alternativas**
> 
> * **Manhã:** Aluguel de bicicletas (Yellow Bike, MacBike) e passeio pelo Vondelpark.
> * **Almoço:** Piquenique no Vondelpark ou almoce no De Kas, um restaurante encantador em uma estufa. 
> * **Tarde:**  Explore o alternativo bairro Noord. Visite o NDSM, um antigo estaleiro transformado em espaço cultural com arte de rua, bares descolados e restaurantes. 
> * **Jantar:** Pllek (ambiente industrial-chique, vista para o rio IJ) ou Noorderlicht (refeições orgânicas em um ambiente aconchegante).
> * **Noite:** Descubra bares alternativos como o Hannekes Boom e o Oedipus Brewing.
> 
> **Dia 4: Mercados & Canais Festivos**
> 
> * **Manhã:** Passeio de barco pelos canais (diversas empresas oferecem tours) para apreciar a arquitetura e as pontes charmosas. 
> * **Almoço:**  Vlaamse Frites (batatas fritas holandesas) no Manneken Pis. 
> * **Tarde:**  Compra de souvenirs e petiscos no Albert Cuyp Market, o maior mercado ao ar livre da Holanda. 
> * **Jantar:** The Pantry (comida caseira holandesa) ou Restaurant Greetje (pratos tradicionais com toques modernos).
> * **Noite:**  Pub crawl pelos bares tradicionais da Red Light District, como o Old Sailor e o Café Belgique.
> 
> **Dia 5: Flores & Heineken Experience**
> 
> * **Manhã:**  Visita ao Bloemenmarkt, o mercado de flores flutuante.
> * **Almoço:**  Hap Hmmm (sanduíches gourmet).
> * **Tarde:**  Heineken Experience (reserve online: [https://www.heinekenexperience.com/](https://www.heinekenexperience.com/)), uma experiência interativa e divertida no mundo da cerveja. 
> * **Jantar:**  Cannibale Royale (hambúrgueres e cervejas) ou Brouwerij ‘t IJ (cervejaria local com opções vegetarianas).
> * **Noite:**  Curta um filme no Pathé Tuschinski, um cinema art déco deslumbrante.
> 
> **Dia 6: Arte Moderna & Despedida**
> 
> * **Manhã:**  Visita ao Moco Museum (Banksy e outros artistas modernos: [https://mocomuseum.com/](https://mocomuseum.com/)).
> * **Almoço:**  Foodtrucks na Museumplein.
> * **Tarde:**  Últimas compras e tempo livre. Relaxe em um café em Rembrandtplein, observe as pessoas e aproveite a atmosfera vibrante.
> * **Jantar:** De Silveren Spiegel (cozinha holandesa sofisticada) ou Toscanini (italiano refinado).
> * **Noite:**   Última oportunidade para curtir a vida noturna de Amsterdam.
> 
> **Dia 7: Partida**
> 
> * Despedida da cidade e deslocamento para o Aeroporto Schiphol (AMS).
> 
> **Transporte:** Amsterdam é uma cidade ideal para explorar a pé ou de bicicleta. Alugue uma bicicleta para uma experiência autêntica e explore no seu ritmo.  O transporte público, como trams, ônibus e metrô, também é eficiente.
> 
> **Custo da viagem:**
> 
> * Passagem aérea (ida e volta): R$ 3.500 - R$ 5.000
> * Hospedagem (6 noites em hotel 3 estrelas): R$ 3.000 - R$ 4.500
> * Alimentação (7 dias): R$ 1.750 - R$ 3.500
> * Transporte local, passeios e atrações: R$ 1.750 - R$ 2.500
> 
> **Total:** R$ 10.000 - R$ 15.500 (estimativa para uma pessoa)
> 
> **Observações:**
> 
> *  A média de custo é uma estimativa, o valor final pode variar de acordo com o estilo de viagem e escolhas pessoais.
> *  Reserve acomodações, passagens aéreas e algumas atrações com antecedência, principalmente na alta temporada.
> *  Aproveite a vibe vibrante de Amsterdam e explore a cidade ao máximo!